# Tutorial on LangChain

LangChain is a powerful framework designed to assist developers in creating applications that integrate with large language models (LLMs) like OpenAI's GPT-4, LLaMA 3.1, and mmany other LLMs. By using LangChain, you can build systems capable of handling complex tasks, maintaining memory across conversations, and chaining responses, allowing for richer interactions.

In this tutorial, we will cover how to:

* Install LangChain and dependencies.
* Understand and use the basic concepts of LangChain.
* Build a simple LangChain application that interacts with GPT.
* Create conversational agents.
* Use memory for conversation history.
* Integrate external data sources, such as APIs.

# 1. Installation

Before we begin using LangChain, you need to install it alongside the required dependencies, like langchain-groq for accessing GROQ.

In [ ]:
!pip install langchain langchain-community langchain-groq duckduckgo-search geopy requests

# 2. Key Concepts in LangChain

LangChain revolves around several core concepts that allow you to build flexible and powerful applications. Here’s a quick overview of each:

## A. LLMs (Large Language Models)
LangChain is built to work with large language models like GPT-4. These models are responsible for generating responses based on the input they receive. By using LangChain, you can control how to query and process responses from these models.

## B. Chains
Chains are the primary building blocks in LangChain. A chain could be a simple sequence of operations, like querying the LLM, or something more complex, involving multiple LLM calls, conditionals, and external APIs.

## C. Agents
Agents are decision-making systems that interact with users and decide the best action to take. An agent can process user queries and choose between calling different APIs, querying the LLM, or performing computations.

## D. Memory
LangChain allows you to store conversation history using the memory module. This feature is particularly useful in building conversational systems where past interactions are relevant to future responses.

# 3. Building a Basic LangChain Application

## Step 1: Set Up the LLM

First, you need to configure GROQ API so that LangChain can communicate with the LLaMA model. You can either set the API key as an environment variable or pass it directly into your code.

In [1]:
groq_api_key = "your_api_key"

To obtain an API key from Groq, follow these steps:

1. Visit the official [Groq website](https://groq.com/).
2. In the top navigation bar, go to the `Developers` section and select [`Start Building`](https://console.groq.com/login). This will direct you to log in to Groq's console.
3. Once logged in, click on the [`API Keys`](https://console.groq.com/keys) tab in the left-hand menu.
4. Click the `Create API Key` button and provide a name for your new key.
5. Copy the generated API key immediately, as it will not be available for copying later. You're all set!

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.8,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=groq_api_key
)

Here, we’re setting the `temperature` to 0.7. The temperature controls the randomness of the output, with lower values generating more focused results and higher values producing more diverse responses.

## Step 2: Create a Prompt Template

Prompt templates in LangChain allow you to build dynamic prompts that are passed to the LLM. Let’s create a template that will generate a story based on user input.

In [3]:
from langchain.prompts import PromptTemplate

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["topic"],
    template="Write a short story about {topic}."
)

This prompt takes one input variable `topic` and generates a story about the topic entered by the user.

## Step 3: Create an LLM Chain

Next, we’ll create an LLM Chain that combines the LLM with our prompt template. This chain will execute the prompt and retrieve the model’s response.

In [ ]:
from langchain import LLMChain

# Create an LLM chain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain with a specific topic
response = chain.run("a brave knight")
print(response)

When you run the code, it will generate a short story about "a brave knight."

# 4. Building a Conversational Agent

## Use LangChain pre-built tools

Agents in LangChain allow your application to make decisions on which actions to take based on the user’s query. For example, an agent might decide to call an external API, retrieve data from a database, or query the LLM.

### Step 1: Define External Tools

Lets select a tool from [LangChain's documentation](https://python.langchain.com/v0.2/docs/integrations/tools/), for this example lets use the [DuckDuckGo](https://python.langchain.com/v0.2/docs/integrations/tools/ddg/) tool as a search function to give the LLM.

First, lets explore the tool!

In [5]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

This is the default name

In [ ]:
search.name

This is the default description

In [ ]:
search.description

This is the default JSON schema of the inputs

In [ ]:
search.args

We can see if the tool should return directly to the user

In [ ]:
search.return_direct

We can call this tool with a dictionary input

In [ ]:
search.invoke({"query": "What is Tuwaiq Academy?"})

Or simply by passing the query

In [ ]:
search.invoke("What is Tuwaiq Academy?")

### Step 2: Set Up the Agent

In [ ]:
from langchain.agents import initialize_agent, AgentType

# List of tools the agent can use
tools = [search]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

The agent uses **ZERO_SHOT_REACT_DESCRIPTION**, which allows it to decide which tool to use based on the input.

### Step 3: Run the Agent

Now that the agent is set up, let’s run it and see how it responds to a query.

In [ ]:
response = agent.run("Find information about SDAIA T5.")
print(response)

The agent will decide to call the search tool and return the search results.

## Create & Use Custom Tools

### Step 1: Define External Tools

Let’s first define a tool that the agent can use. For demonstration purposes, we’ll create a simple function that gets the weather in a City.

For that we will use [open meteo API](https://open-meteo.com/)

In [ ]:
import requests

# We need to define the latitude and longitude for the location, this will be gevin to meteo API to ge the weather
latitude = 35.6895  # Tokyo, Japan latitude
longitude = 139.6917  # Tokyo, Japan longitude

# Open-Meteo API endpoint, we need to specify the latitude and longitude for the desired city in order to use this API
url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"

# Send a GET request to the Open-Meteo API, the response will contain a JSON response containing the needed information or error if one occured
response = requests.get(url)

response

the response was 200 which translated to OK, we will need to get the JSON response. Lets take a look on what the response look like.

In [ ]:
weather_data = response.json()
weather_data

Thats a lot of information, we will pick a few of them.

In [ ]:
# Check if the request was successful, 200 will be translated to OK
if response.status_code == 200:
    # Parse the JSON data from the response
    weather_data = response.json()

    # Extract relevant information from the response
    current_weather = weather_data.get('current_weather', {})
    temperature = current_weather.get('temperature')
    windspeed = current_weather.get('windspeed')
    winddirection = current_weather.get('winddirection')
    weather_time = current_weather.get('time')

    # Print the weather information
    print(f"Current weather in Tokyo:")
    print(f"Temperature: {temperature}°C")
    print(f"Wind Speed: {windspeed} m/s")
    print(f"Wind Direction: {winddirection}°")
    print(f"Time of data: {weather_time}")
else:
    print("Failed to retrieve weather data")

But we can't make the LLM decide what is the latitude and longitude of the city the user queried about, lets use `geopy` to get these information.

First lets create geolocator object and initialize `user_agent`.

In [19]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="LLMexercise")

Now, lets get the latitude and longitude.

In [ ]:
city_name = 'Riyadh Saudi Arabia '
location = geolocator.geocode(city_name)
location

That is great, now lets extract the latitude and longitude.

In [ ]:
latitude = location.latitude
longitude = location.longitude

print(f'Latitude: {latitude}, longitude: {longitude}')

Now lets try using these information with `meteo`, but lets also make a function that does all that and test it.

In [ ]:
import requests
from geopy.geocoders import Nominatim

def get_weather_by_city(city_name: str):
    # Initialize geocoder
    geolocator = Nominatim(user_agent="LLMexercise")

    # Get location data (latitude and longitude) for the city
    location = geolocator.geocode(city_name)

    if location:
        latitude = location.latitude
        longitude = location.longitude
    else:
        return f"City '{city_name}' not found."

    # Open-Meteo API endpoint with the obtained latitude and longitude
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true"

    # Send a GET request to the Open-Meteo API
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON data from the response
        weather_data = response.json()

        # Extract relevant information from the response
        current_weather = weather_data.get('current_weather', {})
        temperature = current_weather.get('temperature')
        windspeed = current_weather.get('windspeed')
        winddirection = current_weather.get('winddirection')
        weather_time = current_weather.get('time')

        # Return the weather information as a formatted string
        return (
            f"Current weather in {city_name}:\n"
            f"Temperature: {temperature}°C\n"
            f"Wind Speed: {windspeed} m/s\n"
            f"Wind Direction: {winddirection}°\n"
            f"Time of data: {weather_time}"
        )
    else:
        return "Failed to retrieve weather data."

city_name = "Riyadh Saudi Arabia"
weather_info = get_weather_by_city(city_name)
print(weather_info)

That is fantastic! now lets give this function to the LLM to use.

In [31]:
from langchain.agents import Tool

weather = Tool(
    name="weather",
    func=get_weather_by_city,
    description="Gets the weather of a city."
)

In [ ]:
weather.name

In [ ]:
weather.description

In [ ]:
weather.args

### Step 2: Set Up the Agent

We can now set up an agent that uses the weather tool. The agent will decide whether to query the LLM or use the weather tool based on the user’s input.

In [35]:
from langchain.agents import initialize_agent, AgentType

# List of tools the agent can use
tools = [weather]

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

The agent uses ZERO_SHOT_REACT_DESCRIPTION, which allows it to decide which tool to use based on the input.

In [ ]:
response = agent.run("What is the weather in Riyadh Saudi Arabia?")
print(response)

Now we have LLM that is capable of checking the weather in Riyadh Saudi Arabia in real-time!

# 5. Using Memory for Conversations

Memory is a key feature when building conversational systems that need to remember past interactions. LangChain allows you to use various types of memory to retain conversation history.

## Using ConversationBufferMemory

### Step 1: Set Up Memory

You can use a memory buffer to store conversation history and include it in future prompts.

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain import ConversationChain

# Initialize summarization-based memory
memory = ConversationBufferMemory()

# Clear memory
memory.clear()

# Create a chain with LLM and buffer memory
chain = ConversationChain(llm=llm, memory=memory)

In this case, the second query (“What color was the dragon?”) will refer back to the dragon mentioned in the first query.

### Step 2: Interact with the LLM with Summarization Memory

In [ ]:
# Start a conversation
res1 = chain.run("Answer with short answer, give me a number between (0 and 9)")
res2 = chain.run("Answer with short answer, give me another number between (10 and 19)")
res3 = chain.run("From the previous conversations. What was these numbers and what is the result of adding them ?")

print(f'First Response: {res1}\nSecond Response: {res2}\nThird Response: {res3}')

## LLM as Memory in LangChain

LLM as memory allows us to query the language model to summarize or remember certain pieces of information from previous conversations. This is particularly useful when you're working with large datasets or conversations where storing all the information might not be practical.

### Step 1: Set Up a Summary-based Memory System

LangChain includes a class that allows the use of summarization memory, where the LLM generates a summary of past interactions.

Let's begin by setting up the ConversationSummaryMemory that uses the LLM to remember interactions.

In [ ]:
from langchain.memory import ConversationSummaryMemory
from langchain import ConversationChain

# Initialize summarization-based memory
summary_memory = ConversationSummaryMemory(llm=llm)

# Clear memory
summary_memory.clear()

# Create a chain with LLM and buffer memory
chain_with_buffer_memory = ConversationChain(llm=llm, memory=summary_memory)

In this case, we use the `ConversationSummaryMemory` module, which will generate summaries of the conversations so that the model can use that summary for context in future interactions.

### Step 2: Interact with the LLM with Summarization Memory

In [ ]:
# Start a conversation
res1 = chain_with_buffer_memory.run("Answer with short answer, give me a number between (0 and 9)")
res2 = chain_with_buffer_memory.run("Answer with short answer, give me another number between (10 and 19)")
res3 = chain_with_buffer_memory.run("From the previous conversations. What was these numbers and what is the result of adding them ?")

print(f'First Response: {res1}\nSecond Response: {res2}\nThird Response: {res3}')

# Conclusion

## By incorporating LLM as memory, LangChain allows for:

* **Enhanced recall** of previous conversations, enabling long-term context in conversations.
* **Dynamic summarization** that uses the LLM itself to remember key points.
* **Efficient memory management**, especially useful for complex, multi-turn interactions where remembering everything is unnecessary.

## Recap of What We’ve Learned:
* **Basic LLM Queries:** Interacting with GPT models using LangChain.
* **Agents:** Creating intelligent agents that make decisions on which actions to take.
* **Memory Buffers:** Storing conversation history using buffers.
* **LLM as Memory:** Summarizing and recalling past interactions using the language model itself.
* **Combining Tools and Memory:** Creating agents that use both memory and external tools for dynamic, real-world tasks.

With LangChain’s memory features, you can build more intelligent and context-aware applications that retain conversation history, summarize large datasets, and combine external data sources effectively.